In [39]:
import pandas as pd
import os

## Loading data

In [40]:
path = os.getcwd()[0:-5] + '\\Data\\kprm-with-salary-update-20231115.json'

df = pd.read_json(path, lines=True)

# Dropping unnecessary columns:
# '6' has been split on 2 columns due to an error during the download,
# while '8' is a duplicate of '9' with minor differences. However, '9' is
# believed to be the correct version.
df.drop(columns=[6,8], inplace=True)

# Renaming columns.
df.columns = ['Ad ID', 'Job title', 'Employer (entity)', 'Employer location', 'Office address', 'Workplace', 'Department', 'Posted on', 'Expiration date', 'Detailed recruitment results', 'Salary (monthly)', 'Vacancies', 'Full-time ratio', 'Recruitment results', 'Description of duties', 'Required level of education', 'Prerequisites', 'Optional requirements', 'Application instructions', 'Number of views']

In [41]:
print(df['Recruitment results'].nunique())

1


In [42]:
# The data contains no active posts at the time of the download:
# the column "Recruitment results" contains only one unique value - "Koniec naboru"
# (recruitment finished).
# Thus, we can drop the column.
df.drop(columns=["Recruitment results"], inplace=True)

## Cleansing

#### Reducing data size
<span style="font-size:1.05em;">We're removing unnecessary text from rows, such as "NR" (an abbreviation for number) in every cell of the "Ad ID" column.</span>

In [43]:
for index, row in df.iterrows():
    if row['Ad ID'][0] == 'N' and row['Ad ID'][2] == ' ':
        df.at[index, 'Ad ID'] = row['Ad ID'][3:]
    
    # Removing "Adres urzędu: " from each row.
    if row['Office address'][12] == ':' and row['Office address'][13] == ' ':
        df.at[index, 'Office address'] = row['Office address'][14:]
    elif row['Office address'][12] == ':':
        df.at[index, 'Office address'] = row['Office address'][13:]
        
    # Removing "Miejsce wykonywania pracy: " from each row.
    if row['Workplace'][25] == ':' and row['Workplace'][26] == ' ':
        df.at[index, 'Workplace'] = row['Workplace'][27:]
    elif row['Workplace'][25] == ':':
        df.at[index, 'Workplace'] = row['Workplace'][26:]

    # Removing "Wykształcenie: " from each row. 
    if row['Required level of education'][13] == ':' and row['Required level of education'][14] == ' ':
        df.at[index, 'Required level of education'] = row['Required level of education'][15:]
    elif row['Required level of education'][13] == ':':
        df.at[index, 'Required level of education'] = row['Required level of education'][14:]
        
    # Removing "Wyniki naboru: " from each row.
    if row['Detailed recruitment results'][14] == ' ' and row['Detailed recruitment results'][13] == ':':
        df.at[index, 'Detailed recruitment results'] = row['Detailed recruitment results'][15:]
    elif row['Detailed recruitment results'][13] == ':':
        df.at[index, 'Detailed recruitment results'] = row['Detailed recruitment results'][14:]
    
    # Removing "Liczba odwiedzin: " from each row.
    # One additional condition since the first character in most entries is a space.
    if row['Required level of education'][0] == ' ':
        if row['Number of views'][18] == ':' and row['Number of views'][19] == ' ':
            df.at[index, 'Number of views'] = row['Number of views'][20:]
        elif row['Number of views'][18] == ':' and row['Number of views'][1].lower() == 'l':
            df.at[index, 'Number of views'] = row['Number of views'][19:]
    else:
        if row['Number of views'][17] == ':' and row['Number of views'][18] == ' ':
            df.at[index, 'Number of views'] = row['Number of views'][19:]
        elif row['Number of views'][16] == ':' and row['Number of views'][0].lower() == 'l':
            df.at[index, 'Number of views'] = row['Number of views'][17:]

        
df.head()

,Ad ID,Job title,Employer (entity),Employer location,Office address,Workplace,Department,Posted on,Expiration date,Detailed recruitment results,Salary (monthly),Vacancies,Full-time ratio,Description of duties,Required level of education,Prerequisites,Optional requirements,Application instructions,Number of views
0,131450,główny specjalista,Ministerstwo Rolnictwa i Rozwoju Wsi w Warszawie,Warszawa,ul. Wspólna 30 00-930 Warszawa,Warszawa,w Wydziale Obsługi Płacowej w Biurze Finansowym,15 grudnia 2023,29 grudnia 2023,anulowano nabór|,"7009,44 zł brutto",1,1,[nalicza wynagrodzenia pracowników oraz zasiłk...,wyższe,[3 lata stażu pracy w obszarze wynagrodzeń i u...,[Wykształcenie wyższe ekonomiczne lub ekonomic...,"[Dokumenty należy złożyć do: 29.12.2023, Decyd...",91
1,131558,księgowy,Powiatowy Inspektorat Weterynarii w Piszu,Pisz,ul. Warszawska 38 12-200 Pisz,Pisz Powiatowy Inspektorat Weterynarii ul. War...,w zespole ds. finansowo-księgowych i administr...,15 grudnia 2023,29 grudnia 2023,anulowano nabór|,nie podano wynagrodzenia,1,1,[Wykonuje zadania księgowego w jednostce budże...,wyższe ekonomiczne,"[doświadczenie zawodowe: co najmniej 1 rok , W...",[komunikatywność],"[Dokumenty należy złożyć do: 29.12.2023, Decyd...",25
2,131502,starszy specjalista,Główny Urząd Nadzoru Budowlanego w Warszawie,Warszawa,Krucza 38/42 00-926 Warszawa,Warszawa ul. Krucza 38/42,w Biurze Organizacyjnym,14 grudnia 2023,29 grudnia 2023,anulowano nabór|,nie podano wynagrodzenia,1,1,[organizuje proces przeprowadzania naborów do ...,wyższe,"[doświadczenie zawodowe: 1,5 roku w pracy zwią...",[Wykształcenie: wyższe prawnicze lub administr...,"[Dokumenty należy złożyć do: 29.12.2023, Decyd...",34
3,131510,inspektor,Powiatowy Inspektorat Nadzoru Budowlanego w Hr...,Hrubieszów,ul. Plac Wolności 13 22-500 Hrubieszów,"Hrubieszów 22-500 Hrubieszów, ul. Plac Wolnośc...",Referat inspekcji i kontroli,13 grudnia 2023,29 grudnia 2023,anulowano nabór|,nie podano wynagrodzenia,1,1,[Prowadzi postępowania administracyjne wynikaj...,średnie budowlane lub architektoniczne,[staż pracy: w administracji publicznej lub w ...,[Wykształcenie: wyższe Wykształcenie: wyższe b...,"[Dokumenty należy złożyć do: 29.12.2023, Decyd...",58
4,131408,główny specjalista,Ministerstwo Rozwoju i Technologii w Warszawie,Warszawa,pl. Trzech Krzyży 3/5 00-507 Warszawa,Warszawa,w Wydziale Wynagrodzeń w Departamencie Budżetu...,13 grudnia 2023,27 grudnia 2023,anulowano nabór|,"od 7009,00 zł do 7447,00 zł brutto",1,1,[prowadzi obsługę płacową dla pracowników zatr...,wyższe ekonomiczne lub inne nieprofilowane wra...,[Doświadczenie zawodowe: minimum 3 lata w obsz...,"[CV i list motywacyjny, Kopie dokumentów potwi...","[Dokumenty należy złożyć do: 27.12.2023, Decyd...",395


#### Making data readable for computers

##### The salary column

<span style="font-size:1.05em;">First, we're checking whether all salaries are given as gross (brutto) or if there are some given as net (netto).</span>

In [44]:
df_netto = df[df['Salary (monthly)'].str.contains('netto')][['Salary (monthly)']]
count_netto = len(df_netto)
print(count_netto)

117


<span style="font-size:1.05em;">Exploring the frequency of different recruitment results.</span>

In [45]:
# Announcements that ended with the employment of a candidate.
success_df = df[df['Detailed recruitment results'].str.contains(r'wyborem kandydatki/kandydata|informacja o zatrudnieniu kandydatki|zatrudnieniem')][['Detailed recruitment results']]
success_count = len(success_df)

In [46]:
# Announcements that ended without employing a candidate.
# Jak skategoryzować ogłoszenia, które w rezultacie mają wpisane "decyzja o rezygnacji kandydata z obejmowanego stanowiska"? Na razie są wrzucone jako fail, aczkolwiek jest to dyskusyjne.

failure_df = df[df['Detailed recruitment results'].str.contains(r'bez wyboru kandydatki/kandydata|nie wybrano|nie zatrudniono|rak aplikacji|rak zgłoszeń|rak ofert|rak kandydatów|rak zapisu|kandydaci zrezygnowali|ikt się nie zgłosił|rak kandydatur|rak wniosków|bez zatrudnienia|kandydat zrezygnował|nie spełniały wymagań|nie wyłoniono|rezygnacji|rezygnacja|kandydat zrezygnował|zrezygnował z|nie zgłosił się|andydaci nie przybyli|zrezygnowali|braku kandydatur|kandydatka zrezygnowała|ezygnacja kandydatki|żadna oferta|rak złożonych ofert|ezygnacja kandydata|nadesłanych ofert|braku kandydatów|braku kandydatur')][['Detailed recruitment results']]
failure_count = len(failure_df)

In [47]:
# Announcements that have been cancelled as a results of an error; they are omitted in our analysis.
error1_df = df[df['Detailed recruitment results'].str.contains(r'łędu|łąd|łędem|łędne|orekta ogłoszenia|łędnie|łędny|łędy w ogłoszeniu|rak informacji|ie uwzględniono|łędna|omyłkowo|omyłka|nie uwwzględniono|przypadkowo|dwukrotnie|podwójnie|razy wprowadzono te same|zdublowane|problem techniczny|miana treści|wymaga korekty')][['Detailed recruitment results']]
error1_count = len(error1_df)

error2_df = df.loc[df['Detailed recruitment results'] == 'anulowano nabór|']
error2_count = len(error2_df)

error_count = error1_count + error2_count
print(f"Ads that led to employment: {success_count}\n"
      f"Ads that did not lead to employment: {failure_count}\n"
      f"Ads cancelled as a result of an error: {error1_count}")

Ads that led to employment: 58659
Ads that did not lead to employment: 43199
Ads cancelled as a result of an error: 464


In [49]:
# A relatively small amount of ads was left, and they're not a subject of our analysis (e.g. a decision to cancel a vacancy or the return of an employee to work).
# From now on, we're going to operate only on ads present in failure_df and success_df.

failure_df = df[df.index.isin(list(failure_df.index))]
success_df = df[df.index.isin(list(success_df.index))]